In [56]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [57]:
#Import calendar and listings df
calendar_df = pd.read_csv('../Data/calendar.csv')
listings_df = pd.read_csv('../Data/listings.csv')

In [69]:
#Helper functions
def get_month(row):
    #returns the month number
    return int(row['date'].split('-')[1])

def get_year(row):
    #returns the year
    return int(row['date'].split('-')[0])
               
def get_host_since(row):
    #returns the year they started hosting
    try:
        host_since = int(row['host_since'].split('-')[0])
    except:
        host_since = np.nan
    return host_since

def get_val(row, column, value):
    #used to fill in the dummy values
    val = 0.0
    try:
        vals = row[column].replace('[','',regex=True).replace("'",'',regex=True) \
                          .replace(']','',regex=True).replace('"','',regex=True) \
                          .replace('{','',regex=True).replace('}','',regex=True) \
                          .split(',')
        if value in vals:
            val = 1.0
    except:
        val = 0.0
    return val

def get_dummy_values(df, column, num_of_dummies = 10):
    #Creates dummie variables from original column
    #Frist create dictionary of unique values
    values_dict = {}
    for unique in df[column].unique():
        for value in unique.replace('[','').replace("'",'') \
                           .replace(']','').replace('"','') \
                           .replace('{','').replace('}','') \
                           .split(','):
            if value in values_dict:
                values_dict[value] = values_dict[value] + 1
            else:
                values_dict[value] = 0
    values_sorted = sorted(values_dict.items(), key=lambda kv: kv[1], reverse = True)
    
    #create new columns
    for value in values_sorted[: num_of_dummies]:
        df[column + '_' + value[0]] = df.apply(lambda row: get_val(row, column, value[0]),axis=1)
    print(values_dict) 
    return

In [59]:
#Change index to allow merging
listings_df = listings_df.rename(index=str, columns={'id': 'listing_id'})
#And then merge
df = pd.merge(calendar_df, listings_df, on = 'listing_id')

In [60]:
#List of columns to drop
columns_to_drop = ['available', 'host_id','host_location','host_acceptance_rate','host_neighbourhood',
                   'host_total_listings_count', 'weekly_price', 'monthly_price',
                   'security_deposit', 'cleaning_fee', 'calendar_updated',
                   'listing_url','last_scraped' ,'scrape_id', 'name', 'summary', 'space', 'description',
                   'experiences_offered', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'zipcode',
                   'neighborhood_overview', 'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
                   'xl_picture_url', 'host_url', 'host_name', 'host_about', 'host_thumbnail_url', 'host_picture_url',
                   'city', 'state', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude',
                   'is_location_exact', 'square_feet', 'has_availability', 'availability_30',
                   'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped',
                  'first_review', 'last_review', 'requires_license', 'license', 'jurisdiction_names', 'price_y',
                  'reviews_per_month']

#Then drop them
df = df.drop(columns = columns_to_drop)

In [61]:
#Now to drop all the rows that do not have a price_x as this is our predictor
df = df.dropna(subset=['price_x'])

In [62]:
#Adding a month and year column and droping the date column
df['month'] = df.apply(lambda row: get_month(row),axis=1)
df['year'] = df.apply(lambda row: get_year(row),axis=1)
df = df.drop(columns = ['date'])

In [63]:
#Convert price to a float and remove non numerical characters
df['price'] = df['price_x'].str.replace("[$, ]","",regex=True).astype("float")
df = df.drop(columns = ['price_x'])

In [64]:
#Turning the host since column into a string and filling in nan with the mean, then drop the original
df['host_since_year'] = df.apply(lambda row: get_host_since(row),axis=1)
df['host_since_year'].fillna(df['host_since_year'].mean(), inplace = True)
df = df.drop(columns = ['host_since'])

In [65]:
#Remove non numerical values from host response rate then convert to numbers.
#Then fill in nan with mean, and put into buckets, finally drop unneeded columns
df['host_response_rate_num'] = df['host_response_rate'].str.replace("%","",regex=True).astype('float')
df['host_response_rate_num'].fillna(df['host_response_rate_num'].mean(), inplace = True)
df['host_response_rate_buckets'] = pd.qcut(df['host_response_rate_num'], 5, labels=False
                                                                       , duplicates='drop')
df = df.drop(columns = ['host_response_rate', 'host_response_rate_num'])

In [66]:
#Fill in the missing values for host listing count with the mean
df['host_listings_count'].fillna(df['host_listings_count'].mean(), inplace = True)

In [71]:
#Create dummy variables for host verifications and drop original
get_dummy_values(df, 'host_verifications')
df = df.drop(columns = ['host_verifications'])

In [72]:
#Fill in the missing values for bathroom, bedrooms, and beds with mode
df['bathrooms'] = df['bathrooms'].fillna(df['bathrooms'].mode()[0])
df['bedrooms'] = df['bedrooms'].fillna(df['bedrooms'].mode()[0])
df['beds'] = df['beds'].fillna(df['beds'].mode()[0])

In [74]:
#Fill in the missing values for review scores with mean
review_scores_columns = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                         'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                         'review_scores_value']

for column in review_scores_columns:
    df[column.fillna(df[column].mean(), inplace = True)]

AttributeError: 'str' object has no attribute 'fillna'

In [73]:
df.sample(10)

,listing_id,host_response_time,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,property_type,room_type,accommodates,...,host_verifications_email,host_verifications_ phone,host_verifications_ reviews,host_verifications_ facebook,host_verifications_ jumio,host_verifications_ kba,host_verifications_ linkedin,host_verifications_ google,host_verifications_ manual_offline,host_verifications_phone
817050,2471659,within a few hours,f,1.0,t,f,Rainier Valley,House,Entire home/apt,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1133831,10332096,NaN,f,1.0,t,f,Lake City,House,Private room,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1193476,6438099,within an hour,f,5.0,t,t,Lake City,House,Private room,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1374749,6043482,NaN,f,1.0,t,t,Other neighborhoods,Apartment,Private room,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
398183,9634448,within an hour,f,1.0,t,t,Downtown,Condominium,Entire home/apt,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
922523,1386306,within an hour,t,5.0,t,t,Other neighborhoods,House,Private room,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
282946,3770248,within a few hours,f,34.0,t,t,University District,Apartment,Entire home/apt,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
460402,6482984,within an hour,t,1.0,t,t,Downtown,Bed & Breakfast,Private room,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
779123,6796066,within a few hours,f,1.0,t,t,Beacon Hill,House,Private room,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
302431,4623541,within a day,f,1.0,t,t,Central Area,House,Entire home/apt,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
df['amenities'].sample(5)

635423     {TV,"Cable TV",Internet,"Wireless Internet",Ki...
1316781    {Internet,"Wireless Internet",Kitchen,"Free Pa...
537607     {"Cable TV","Wireless Internet","Air Condition...
355325     {TV,"Wireless Internet",Heating,"Smoke Detecto...
119598     {"Cable TV","Wireless Internet",Kitchen,Heatin...
Name: amenities, dtype: object